In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# 基础路径
base_path = Path(r"C:\Users\田\Desktop\python实操\kaggle\Natural Language Processing with Disaster Tweets")  # 你的原始路径

In [3]:
sample_submission = pd.read_csv(base_path / 'sample_submission.csv')
test = pd.read_csv(base_path / 'test.csv')
train = pd.read_csv(base_path / 'train.csv')

In [3]:
sample_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [4]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


<big>autogluon<big>

In [4]:
from sklearn.model_selection import train_test_split
xtrain_data, xtest_data = train_test_split(train, test_size=0.2, random_state=42)

In [4]:
%%time
from autogluon.tabular import TabularPredictor


# 正确的预设名（使用你的版本支持的）
predictor = TabularPredictor(
    label='target',  # 替换为你的标签列名
    path='./agModels',
    # verbosity=1,#0,1,2,3,4,    
    eval_metric='f1',     # 'balanced_accuracy',mae,f1_macro,roc_auc,    
    problem_type='binary',    # 'multiclass',binary,regression,    
)

# 关键：使用 'good_quality' 但关闭所有高级功能
predictor.fit(
    train_data=xtrain_data,    
    presets='best', #  ,extreme，best，high，good，medium,
    
    num_bag_folds=0,      # 关闭bagging
    num_stack_levels=0,   # 关闭stacking    
    # auto_stack = True    ,  # 自动堆叠
    
    
    included_model_types=['XGB',],  
    # time_limit=100,  # 先试5分钟   
)

Preset alias specified: 'extreme' maps to 'extreme_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       19.02 GB / 31.28 GB (60.8%)
Disk Space Avail:   191.01 GB / 452.88 GB (42.2%)
Presets specified: ['extreme']
`extreme` preset uses a dynamic portfolio based on dataset size...
	Detected data size: small (<=30000 samples), using `zeroshot_2025_tabfm` portfolio.
		Note: `zeroshot_2025_tabfm` portfolio requires a CUDA compatible GPU for best performance.
		Make sure you have all the relevant dependencies installed: `pip install autogluon.tabular[tabarena]`.
		It is strongly recommended to use a machine with 64+ GB memory and a CUDA compatible GPU with 32+ GB vRAM when using this preset. 
		This portfolio will download foundation model weights from HuggingFace during t

[1000]	valid_set's binary_logloss: 0.512722	valid_set's f1: 0.6875
[2000]	valid_set's binary_logloss: 0.551765	valid_set's f1: 0.701461
[3000]	valid_set's binary_logloss: 0.592191	valid_set's f1: 0.702929


	0.7083	 = Validation score   (f1)
	25.09s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: TabM_r184 ... Training model for up to 3571.09s of the 3571.09s of remaining time.
	Fitting with cpus=16, gpus=1, mem=7.0/18.7 GB
	0.703	 = Validation score   (f1)
	181.37s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: TabM_r69 ... Training model for up to 3389.34s of the 3389.34s of remaining time.
	Fitting with cpus=16, gpus=1, mem=8.9/17.9 GB
	0.7056	 = Validation score   (f1)
	198.64s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: XGBoost_r171 ... Training model for up to 3190.30s of the 3190.30s of remaining time.
	Fitting with cpus=16, gpus=0
	0.6823	 = Validation score   (f1)
	18.85s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: TabM_r52 ... Training model for up to 3171.37s of the 3171.37s of remaining time.
	Fitting with cpus=16, gpus=1, mem=10.7/17.6 GB
	0.7075	 = Validation score   (f1)
	325.09s	 = Training   

[1000]	valid_set's binary_logloss: 0.514794	valid_set's f1: 0.698152


	0.7037	 = Validation score   (f1)
	4.21s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost_r40 ... Training model for up to 2574.45s of the 2574.45s of remaining time.
	Fitting with cpus=16, gpus=0
	0.6908	 = Validation score   (f1)
	8.6s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM_r11 ... Training model for up to 2565.80s of the 2565.80s of remaining time.
	Fitting with cpus=16, gpus=0, mem=0.0/15.7 GB


[1000]	valid_set's binary_logloss: 0.548674	valid_set's f1: 0.578704
[2000]	valid_set's binary_logloss: 0.531804	valid_set's f1: 0.659529
[3000]	valid_set's binary_logloss: 0.527415	valid_set's f1: 0.705155
[4000]	valid_set's binary_logloss: 0.524514	valid_set's f1: 0.712551
[5000]	valid_set's binary_logloss: 0.522447	valid_set's f1: 0.704453


	0.714	 = Validation score   (f1)
	10.24s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: TabM_r49 ... Training model for up to 2555.38s of the 2555.37s of remaining time.
	Fitting with cpus=16, gpus=1, mem=10.6/15.6 GB
	0.7069	 = Validation score   (f1)
	122.55s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 2432.36s of remaining time.
	Ensemble Weights: {'TabM_r191': 0.6, 'LightGBM_r11': 0.4}
	0.7214	 = Validation score   (f1)
	0.14s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1167.84s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 2708.0 rows/s (609 batch size)
Enabling decision threshold calibration (calibrate_decision_threshold='auto', metric is valid, problem_type is 'binary')
Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained sear

CPU times: total: 20min 54s
Wall time: 19min 45s


In [ ]:
'XGB','GBM','RF','XT','FASTAI'

In [ ]:
'TABM','GBM','XGB',

In [5]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.721443,f1,0.224890,176.440517,0.001993,0.139832,2,True,12
1,TabM_r191,0.718254,f1,0.207896,166.057215,0.207896,166.057215,1,True,7
2,LightGBM_r11,0.713996,f1,0.015002,10.243470,0.015002,10.243470,1,True,10
3,LightGBM_r33,0.708333,f1,0.021110,25.087075,0.021110,25.087075,1,True,1
4,TabM,0.708249,f1,0.209465,100.532593,0.209465,100.532593,1,True,6
5,TabM_r52,0.707510,f1,0.321146,325.090560,0.321146,325.090560,1,True,5
6,TabM_r49,0.706861,f1,0.254720,122.549969,0.254720,122.549969,1,True,11
7,TabM_r69,0.705637,f1,0.284041,198.637933,0.284041,198.637933,1,True,3
8,LightGBM_r21,0.703704,f1,0.012002,4.212158,0.012002,4.212158,1,True,8
9,TabM_r184,0.703030,f1,0.282282,181.371925,0.282282,181.371925,1,True,2


In [6]:
predictor.evaluate(xtest_data)

{'f1': 0.7403708987161198,
 'accuracy': 0.7609980302035456,
 'balanced_accuracy': 0.7659786399071975,
 'mcc': 0.5261529026122175,
 'roc_auc': 0.8451569568390729,
 'precision': 0.6892430278884463,
 'recall': 0.7996918335901386}

In [7]:
%%time
predictor.feature_importance(xtest_data)

Computing feature importance via permutation shuffling for 4 features using 1523 rows with 5 shuffle sets...
	19.47s	= Expected runtime (3.89s per shuffle set)
	8.0s	= Actual runtime (Completed 5 of 5 shuffle sets)


CPU times: total: 38.3 s
Wall time: 8 s


,importance,stddev,p_value,n,p99_high,p99_low
text,0.203570,0.005707,7.401976e-08,5,0.215320,0.191821
keyword,0.102147,0.011388,1.823644e-05,5,0.125596,0.078699
location,0.006447,0.004052,1.181500e-02,5,0.014790,-0.001896
id,0.002192,0.001052,4.794186e-03,5,0.004358,0.000026


In [27]:
del sub

In [14]:
sub=test[['id']].copy()

In [15]:
predictions = predictor.predict(test,)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Output()

C:\anaconda\envs\py312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


In [16]:
sub=test[['id']].copy()

In [17]:
sub['target']=predictions

In [22]:
sub

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [23]:
sub.to_csv('kaggel_nlp_getting_started-0106-05.csv',index=False)

In [ ]:
bset:0.75543

In [ ]:
extream:0.76861,19min 22s

In [ ]:
多模态：0.77965

<big>传统ML，counter，TF-IDF，文本向量化<big>

In [9]:
%%time
# ==============================
# 灾难推文分类 - 四种方案对比
# 取消交叉验证，直接训练
# ==============================

import pandas as pd
import numpy as np
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

# ==============================
# 1. 数据预处理
# ==============================
print("1. 数据预处理...")

def clean_text(tweet):
    tweet = str(tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet.lower()

df_train = train
df_test = test

df_train['text_cleaned'] = df_train['text'].apply(clean_text)
df_test['text_cleaned'] = df_test['text'].apply(clean_text)

X = df_train['text_cleaned'].values
y = df_train['target'].values
X_test = df_test['text_cleaned'].values

print(f"训练集: {len(X)}条, 测试集: {len(X_test)}条")

# ==============================
# 2. 方案1: CountVectorizer
# ==============================
print("\n" + "="*50)
print("方案1: CountVectorizer + 逻辑回归")
print("="*50)

count_vectorizer = CountVectorizer(max_features=3000, stop_words='english')
X_count = count_vectorizer.fit_transform(X)
X_test_count = count_vectorizer.transform(X_test)

count_model = LogisticRegression(max_iter=1000, random_state=42)
count_model.fit(X_count, y)
y_pred_count = count_model.predict(X_test_count)

print("✅ 完成")

# ==============================
# 3. 方案2: TF-IDF
# ==============================
print("\n" + "="*50)
print("方案2: TF-IDF + 逻辑回归")
print("="*50)

tfidf_vectorizer = TfidfVectorizer(max_features=3000, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(X)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

tfidf_model = LogisticRegression(max_iter=1000, random_state=42)
tfidf_model.fit(X_tfidf, y)
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

print("✅ 完成")

# ==============================
# 4. 方案3: 随机森林
# ==============================
print("\n" + "="*50)
print("方案3: TF-IDF + 随机森林")
print("="*50)

rf_model = RandomForestClassifier(n_estimators=50, max_depth=15, random_state=42, n_jobs=-1)
rf_model.fit(X_tfidf, y)
y_pred_rf = rf_model.predict(X_test_tfidf)

print("✅ 完成")

# ==============================
# 5. 方案4: BERT向量化
# ==============================
print("\n" + "="*50)
print("方案4: BERT向量化 + 逻辑回归")
print("="*50)

print("加载BERT模型...")
MODEL_NAME = r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)
bert_model.eval()

# 简化BERT向量提取
def get_bert_vectors(texts, batch_size=16):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch.tolist(), padding=True, truncation=True, max_length=64, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = bert_model(**inputs)
        
        vectors.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())
        
    return np.vstack(vectors)

# 只处理前2000条，加快速度
print("提取BERT向量（简化版）...")
sample_size = min(2000, len(X), len(X_test))
X_bert_sample = get_bert_vectors(X[:sample_size])
X_test_bert_sample = get_bert_vectors(X_test[:sample_size])

# 训练
bert_model_lr = LogisticRegression(max_iter=1000, random_state=42)
bert_model_lr.fit(X_bert_sample, y[:sample_size])

# 预测
y_pred_bert = np.zeros(len(X_test))
y_pred_bert[:sample_size] = bert_model_lr.predict(X_test_bert_sample)
y_pred_bert[sample_size:] = 0  # 其他用0填充

print("✅ 完成")

# ==============================
# 6. 生成四种预测结果
# ==============================
print("\n" + "="*50)
print("6. 生成预测结果")
print("="*50)

# 保存每种方法的预测
submissions = {
    'count': sample_submission.copy(),
    'tfidf': sample_submission.copy(),
    'rf': sample_submission.copy(),
    'bert': sample_submission.copy()
}

# 确保长度一致
for i, (key, pred) in enumerate([('count', y_pred_count), ('tfidf', y_pred_tfidf), 
                                  ('rf', y_pred_rf), ('bert', y_pred_bert)]):
    if len(pred) > len(submissions[key]):
        submissions[key]['target'] = pred[:len(submissions[key])]
    else:
        # 补充0
        padded = np.pad(pred, (0, len(submissions[key])-len(pred)), 'constant')
        submissions[key]['target'] = padded

# 保存四种结果
# submissions['count'].to_csv('count_submission.csv', index=False)
# submissions['tfidf'].to_csv('tfidf_submission.csv', index=False)
# submissions['rf'].to_csv('rf_submission.csv', index=False)
# submissions['bert'].to_csv('bert_submission.csv', index=False)

print("四种结果已保存:")


1. 数据预处理...
训练集: 7613条, 测试集: 3263条

方案1: CountVectorizer + 逻辑回归
✅ 完成

方案2: TF-IDF + 逻辑回归
✅ 完成

方案3: TF-IDF + 随机森林
✅ 完成

方案4: BERT向量化 + 逻辑回归
加载BERT模型...
提取BERT向量（简化版）...
✅ 完成

6. 生成预测结果
四种结果已保存:
CPU times: total: 9.33 s
Wall time: 8.9 s


<big>sft<big>

In [6]:
%%time
# ==============================
# 灾难推文分类 - SFT风格BERT
# 保持BERT模型，但用生成式训练方式
# ==============================

import pandas as pd
import numpy as np
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,  # 用回分类模型
    get_linear_schedule_with_warmup
)
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

# ==============================
# 1. 配置参数
# ==============================
SEED = 42
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 2
LR = 2e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 使用BERT模型
MODEL_NAME = r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'

# ==============================
# 2. 数据处理函数
# ==============================
def clean_text(tweet):
    """清理文本"""
    tweet = str(tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

def format_sft_prompt(text, label=None):
    """
    格式化指令提示 - 为BERT调整
    """
    instruction = "判断以下推文是否描述真实灾难事件："
    prompt = f"{instruction} {text}"
    return prompt

# ==============================
# 3. 数据集类
# ==============================
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = format_sft_prompt(str(self.texts[idx]))
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            # 测试数据没有标签
            item['labels'] = torch.tensor(0, dtype=torch.long)  # 占位符
            
        return item

# ==============================
# 4. 训练和评估函数
# ==============================
def train_epoch(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0
    
    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        if (batch_idx + 1) % 10 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            print(f"  Batch {batch_idx+1}/{len(dataloader)}, Loss: {avg_loss:.4f}")
    
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_targets, all_predictions, average='macro')
    return f1, all_predictions

# ==============================
# 5. 主训练流程
# ==============================
def main():
    print("🚀 开始灾难推文分类训练")
    print(f"设备: {DEVICE}")
    print(f"模型: {MODEL_NAME}")
    
    # 加载数据
    print("\n📊 加载和预处理数据...")
    df_train = train
    df_test = test
    
    # 清理文本
    df_train['text_cleaned'] = df_train['text'].apply(clean_text)
    df_test['text_cleaned'] = df_test['text'].apply(clean_text)
    
    # 初始化tokenizer和模型
    print(f"\n🔄 加载tokenizer和模型")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # 关键修复：为BERT设置padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
    # 加载模型
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )
    model.resize_token_embeddings(len(tokenizer))
    model = model.to(DEVICE)
    
    print(f"✅ 模型加载完成")
    
    # 准备数据
    from sklearn.model_selection import train_test_split
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df_train['text_cleaned'].values,
        df_train['target'].values,
        test_size=0.2,
        random_state=SEED,
        stratify=df_train['target']
    )
    
    # 创建数据集
    train_dataset = TweetDataset(train_texts, train_labels, tokenizer, MAX_LEN)
    val_dataset = TweetDataset(val_texts, val_labels, tokenizer, MAX_LEN)
    test_dataset = TweetDataset(df_test['text_cleaned'].values, None, tokenizer, MAX_LEN)
    
    # 创建数据加载器
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # 优化器
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=int(total_steps * 0.1),
        num_training_steps=total_steps
    )
    
    # 训练循环
    print(f"\n🎯 开始训练，共 {EPOCHS} 个epoch")
    best_f1 = 0
    
    for epoch in range(EPOCHS):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        print(f"{'='*50}")
        
        # 训练
        train_loss = train_epoch(model, train_loader, optimizer, scheduler)
        print(f"训练损失: {train_loss:.4f}")
        
        # 评估
        f1_score_val, _ = evaluate_model(model, val_loader)
        print(f"验证集F1分数: {f1_score_val:.4f}")
        
        if f1_score_val > best_f1:
            best_f1 = f1_score_val
            torch.save(model.state_dict(), f"best_model_epoch{epoch+1}.pth")
            print(f"✅ 保存最佳模型，F1: {best_f1:.4f}")
    
    # 测试集预测
    print(f"\n🔮 在测试集上生成预测...")
    if EPOCHS > 0:
        model.load_state_dict(torch.load(f"best_model_epoch{EPOCHS}.pth"))
    
    model.eval()
    test_predictions = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            test_predictions.extend(predictions.cpu().numpy())
    
    # 生成提交文件
    print(f"\n💾 生成提交文件...")
    submission = sample_submission.copy()
    submission['target'] = test_predictions[:len(submission)]
    
    submission.to_csv('bert_sft_submission.csv', index=False)
    print(f"✅ 提交文件已保存: bert_sft_submission.csv")
    print(f"📈 最佳F1分数: {best_f1:.4f}")
    
    return submission

# ==============================
# 6. 运行训练
# ==============================
if __name__ == "__main__":
    try:
        submission = main()
        print("\n✅ 训练完成!")
        
        # 验证前几个预测
        print("\n🔍 验证预测结果:")
        print(f"测试集大小: {len(submission)}")
        print(f"预测分布: 0={sum(submission['target']==0)}, 1={sum(submission['target']==1)}")
        
    except Exception as e:
        print(f"\n❌ 错误: {e}")
        import traceback
        traceback.print_exc()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 开始灾难推文分类训练
设备: cuda
模型: C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased

📊 加载和预处理数据...

🔄 加载tokenizer和模型
✅ 模型加载完成

🎯 开始训练，共 2 个epoch

Epoch 1/2
  Batch 10/191, Loss: 0.6845
  Batch 20/191, Loss: 0.6714
  Batch 30/191, Loss: 0.6396
  Batch 40/191, Loss: 0.6000
  Batch 50/191, Loss: 0.5700
  Batch 60/191, Loss: 0.5487
  Batch 70/191, Loss: 0.5285
  Batch 80/191, Loss: 0.5154
  Batch 90/191, Loss: 0.5154
  Batch 100/191, Loss: 0.5096
  Batch 110/191, Loss: 0.5018
  Batch 120/191, Loss: 0.4936
  Batch 130/191, Loss: 0.4858
  Batch 140/191, Loss: 0.4796
  Batch 150/191, Loss: 0.4766
  Batch 160/191, Loss: 0.4713
  Batch 170/191, Loss: 0.4677
  Batch 180/191, Loss: 0.4664
  Batch 190/191, Loss: 0.4636
训练损失: 0.4643
验证集F1分数: 0.8393
✅ 保存最佳模型，F1: 0.8393

Epoch 2/2
  Batch 10/191, Loss: 0.3331
  Batch 20/191, Loss: 0.3515
  Batch 30/191, Loss: 0.3366
  Batch 40/191, Loss: 0.3315
  Batch 50/191, Loss: 0.3362
  Batch 60/191, Loss: 0.3342
  Batch 70/191, Loss: 0.3372
  Batch 

In [ ]:
Score: 0.83205

<big>transformer，all<big>

<big>风控相关<big>

In [11]:
%%time
# ==============================
# 灾难推文分类
# ==============================

import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

# ------------------------------
# 1. 配置
# ------------------------------
SEED = 42
K = 2
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 2
LR = 2e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



def clean(tweet):
    tweet = str(tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

# ------------------------------
# 2. 数据加载
# ------------------------------
print("Loading data...")
df_train = train
df_test = test

print("Cleaning text...")
df_train['text_cleaned'] = df_train['text'].apply(clean)
df_test['text_cleaned'] = df_test['text'].apply(clean)

# ------------------------------
# 3. Dataset类
# ------------------------------
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        

        return {
            'input_ids': encoding['input_ids'].flatten().long(),  # 添加.long()
            'attention_mask': encoding['attention_mask'].flatten().long(),  # 添加.long()
            'label': torch.tensor(label, dtype=torch.long)
        }

# ------------------------------
# 4. 训练函数
# ------------------------------
def train_model(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        optimizer.zero_grad()
        
        # 关键修改：确保数据类型正确
        input_ids = batch['input_ids'].to(DEVICE).long()  # 添加.long()
        attention_mask = batch['attention_mask'].to(DEVICE).long()  # 添加.long()
        labels = batch['label'].to(DEVICE).long()  # 添加.long()
        
        # 调试：打印数据类型
        if hasattr(train_model, 'debug_printed') is False:
            print(f"调试信息 - 输入数据类型:")
            print(f"input_ids: {input_ids.dtype}, shape: {input_ids.shape}")
            print(f"attention_mask: {attention_mask.dtype}, shape: {attention_mask.shape}")
            print(f"labels: {labels.dtype}, shape: {labels.shape}")
            train_model.debug_printed = True
        
        outputs = model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    
    return total_loss / len(dataloader)

def eval_model(model, dataloader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for batch in dataloader:
            # 同样确保数据类型
            input_ids = batch['input_ids'].to(DEVICE).long()
            attention_mask = batch['attention_mask'].to(DEVICE).long()
            labels = batch['label'].to(DEVICE).long()
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
            targets.extend(labels.cpu().numpy())
    return f1_score(targets, preds, average='macro')

# ------------------------------
# 5. 主训练流程
# ------------------------------
print("\nInitializing tokenizer and model...")

# 使用标准BERT模型
model_name = r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'


# model_path = r'C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"✓ Tokenizer loaded: {type(tokenizer).__name__}")

skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)
all_test_preds = []

all_test_logits = []  # 新增：用于存储每个 fold 的 logits，风控修改


for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, df_train['target'])):
    print(f"\n{'='*20} Fold {fold + 1}/{K} {'='*20}")
    
    # 准备数据
    train_texts = df_train.iloc[trn_idx]['text_cleaned'].values
    train_labels = df_train.iloc[trn_idx]['target'].values
    val_texts = df_train.iloc[val_idx]['text_cleaned'].values
    val_labels = df_train.iloc[val_idx]['target'].values
    
    train_dataset = TweetDataset(train_texts, train_labels, tokenizer, MAX_LEN)
    val_dataset = TweetDataset(val_texts, val_labels, tokenizer, MAX_LEN)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # 加载模型
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    ).to(DEVICE)
    
    optimizer = AdamW(model.parameters(), lr=LR, eps=1e-8)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    
    # 训练循环
    best_f1 = 0
    for epoch in range(EPOCHS):
        print(f"  Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_model(model, train_loader, optimizer, scheduler)
        val_f1 = eval_model(model, val_loader)
        print(f"    Train Loss: {train_loss:.4f}, Val F1: {val_f1:.4f}")
        if val_f1 > best_f1:
            best_f1 = val_f1
    
    # 预测测试集
    test_dataset = TweetDataset(df_test['text_cleaned'].values, [0]*len(df_test), tokenizer, MAX_LEN)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model.eval()
    
    #风控修改
    fold_logits = []  # 改为保存 logits
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(DEVICE).long()
            attention_mask = batch['attention_mask'].to(DEVICE).long()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            fold_logits.append(logits.cpu().numpy())  # 保存原始 logits
    
    # 合并所有 batch 的 logits
    fold_logits = np.vstack(fold_logits)  # shape: [n_test, 2]
    all_test_logits.append(fold_logits)

    print(f"  Fold {fold + 1} completed. Best Val F1: {best_f1:.4f}")

# ------------------------------
# 6. 生成提交文件
# ------------------------------
print("\nGenerating submission...")
final_preds = np.round(np.mean(all_test_preds, axis=0)).astype(int)
submission = sample_submission.copy()
submission['target'] = final_preds
print("Submission generated successfully!")

Loading data...
Cleaning text...

Initializing tokenizer and model...
✓ Tokenizer loaded: BertTokenizerFast

==================== Fold 1/2 ====================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Epoch 1/2
调试信息 - 输入数据类型:
input_ids: torch.int64, shape: torch.Size([32, 128])
attention_mask: torch.int64, shape: torch.Size([32, 128])
labels: torch.int64, shape: torch.Size([32])
    Train Loss: 0.4774, Val F1: 0.8189
  Epoch 2/2
    Train Loss: 0.3338, Val F1: 0.8278


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\田\.cache\modelscope\hub\models\google-bert\bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Fold 1 completed. Best Val F1: 0.8278

==================== Fold 2/2 ====================
  Epoch 1/2
    Train Loss: 0.4584, Val F1: 0.8085
  Epoch 2/2
    Train Loss: 0.3422, Val F1: 0.8345
  Fold 2 completed. Best Val F1: 0.8345

Generating submission...
Submission generated successfully!
CPU times: total: 5min 38s
Wall time: 5min 55s


C:\anaconda\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\anaconda\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
<timed exec>:216: RuntimeWarning: invalid value encountered in cast


In [42]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [43]:
submission.to_csv('kaggel_nlp_getting_started-0125-04.csv', index=False)

In [44]:
0.83879

0.83879

<big>将 NLP 模型输出转化为“风控可用特征”  
展示该特征如何用于下游风控（示意）<big>

In [12]:
# ==============================
# 新增：生成风控特征表
# ==============================

# 计算平均预测概率（soft voting）
avg_logits = np.mean(all_test_logits, axis=0)  # shape: [n_test, 2]
y_pred_proba = torch.softmax(torch.tensor(avg_logits), dim=1).numpy()  # 转为概率

# 构造风控特征表
risk_features = pd.DataFrame({
    'tweet_id': df_test['id'].values,
    'user_id': [f"user_{i}" for i in range(len(df_test))],  # 模拟用户ID
    'text_risk_score': y_pred_proba[:, 1],  # 灾难类概率
    'is_high_risk_content': (y_pred_proba[:, 1] > 0.7).astype(int)
})

print("\n✅ 文本风险特征表（可接入风控系统）:")
display(risk_features.head())

# （可选）模拟账号主表融合示例
account_demo = pd.DataFrame({
    'user_id': [f"user_{i}" for i in range(min(100, len(df_test)))],
    'post_count': np.random.poisson(5, min(100, len(df_test))),
    'login_days': np.random.randint(1, 30, min(100, len(df_test)))
})
merged = account_demo.merge(
    risk_features[['user_id', 'text_risk_score']], 
    on='user_id', 
    how='left'
).fillna(0)
print("\n✅ 账号主表 + 文本风险特征（风控模型输入示例）:")
display(merged.head())


✅ 文本风险特征表（可接入风控系统）:


,tweet_id,user_id,text_risk_score,is_high_risk_content
0,0,user_0,0.958550,1
1,2,user_1,0.933660,1
2,3,user_2,0.963134,1
3,9,user_3,0.975063,1
4,11,user_4,0.971232,1



✅ 账号主表 + 文本风险特征（风控模型输入示例）:


,user_id,post_count,login_days,text_risk_score
0,user_0,3,1,0.958550
1,user_1,5,9,0.933660
2,user_2,6,11,0.963134
3,user_3,4,9,0.975063
4,user_4,4,28,0.971232


In [10]:
# 注：在真实业务中，'user_id' 会来自用户行为日志，此处为演示模拟